# SQLAlchemy

This section overviews popular library that provides a tools to operate with database `SQLAlchemy` python library. Check [main page](https://www.sqlalchemy.org/).

In [1]:
import sqlalchemy

## Engine

An engine is a Python object that implements interaction with the database.

For details check:

- [Establishing Connectivity - the Engine](https://docs.sqlalchemy.org/en/20/tutorial/engine.html#tutorial-engine) tutorial.
- [Engine object](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Engine) reference.
- [`create_engine`](https://docs.sqlalchemy.org/en/20/core/engines.html#sqlalchemy.create_engine) function reference.

---

The following cell creates inmemory `sqlite` engine:

In [2]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:")
engine

Engine(sqlite+pysqlite:///:memory:)

Now, to prove that it works as a regular `sqlite` query, retrieve the version of the database.

In [3]:
with engine.connect() as conn:
    ans = conn.execute(sqlalchemy.text("SELECT sqlite_version();"))
ans.fetchall()

[('3.37.2',)]

### Log DB commands

To check what exact commands where executed by the `SQLAlchemy` you can set `echo=True` argument to the `Engine`. After that every command executed by SQLAlchmy will be displayed in the corresponding logger. 

---

The following cell executes some SQL and extracts its result.

In [4]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=True)
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT 'hello world'"))
    result.all()

2025-01-13 18:34:07,420 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-13 18:34:07,421 INFO sqlalchemy.engine.Engine SELECT 'hello world'
2025-01-13 18:34:07,422 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ()
2025-01-13 18:34:07,422 INFO sqlalchemy.engine.Engine ROLLBACK


The corresponding records are shown in the standard output.

## Metadata

Metadata contains information that defines the structure of the database: tables, relationships between tables, and table schemas. Check the more in:

- [Corresponding page of the official documentation](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-working-with-metadata).
- [Special page](sqlalchemy/metadata.ipynb).

---

Following code creates `MetaData`.

In [5]:
metadata = sqlalchemy.MetaData()
metadata

MetaData()

Since it should be a representation of a database in Python code, you can easily create tables using it. The folowing cell adds `test_table` and displays awaible tables.

In [6]:
sqlalchemy.Table(
    "test_table",
    metadata,
    sqlalchemy.Column("id", sqlalchemy.Integer)
)
metadata.tables

FacadeDict({'test_table': Table('test_table', MetaData(), Column('id', Integer(), table=<test_table>), schema=None)})

Apply all changes to the specified database by using `metadata.create_all(<engine for the database>)`.

In [7]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:")
metadata.create_all(engine)

After that, the schema of the database should match the schema created in SQLAlchemy.

In [9]:
with engine.connect() as conn:
    res = conn.execute(
        sqlalchemy.text(
            "SELECT name FROM sqlite_master WHERE type='table'"
        )
    )
    for row in res:
        print(row)

('test_table',)
